In [1]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 817 kB 959 kB/s eta 0:00:01
     |████████████████████████████████| 923 kB 5.6 MB/s eta 0:00:01
     |████████████████████████████████| 525 kB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 174 kB 5.7 MB/s eta 0:00:01
     |████████████████████████████████| 291 kB 10.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 5.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 8.4 MB/s eta 0:00:01
     |████████████████████████████████| 390 kB 5.8 MB/s eta 0:00:01
     |████████████████████████████████| 14.0 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 407 kB 6.2 MB/s eta 0:00:01
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
     |████████████████████████████████| 115 kB 9.2 MB/s eta 0:00:01
     |████████████████████████████████| 311 kB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 291 kB 9

In [88]:
import os 

from IPython.display import display, Markdown, Latex
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

from utils import set_keys, load_files

In [64]:
DB_PATH = "choreo_docs.db"

# set OPENAI KEY
set_keys()

In [103]:
question = "how to fix endpoint.yaml missing error"

In [89]:
# first attempt to use the LLM directly 
prompt = f"""You are an assistant to the Choreo users, to help them to use Choreo platform by answering them on how to use Choreo.
Question: {question}
"""

llm = ChatOpenAI(model_name='gpt-3.5-turbo')
answer = llm.invoke(prompt).content

display(Markdown(answer))

If you are encountering an "endpoint.yaml missing" error, it means that the configuration file for your endpoint is not present in the specified location. 

To fix this error, you will need to create a new endpoint.yaml file or locate the existing file and make sure it is in the correct directory. 

Here are the steps to create a new endpoint.yaml file:
1. Open a text editor or IDE on your computer.
2. Create a new file and save it as "endpoint.yaml".
3. Add the necessary configuration details for your endpoint, such as the endpoint URL, authentication details, headers, etc.
4. Save the file in the directory where Choreo is looking for the endpoint.yaml file.

If you already have an existing endpoint.yaml file but it is not being recognized, make sure it is saved in the correct directory and that the file is properly formatted. 

After creating or locating the endpoint.yaml file, restart Choreo and the error should be resolved. If you continue to encounter issues, you may need to check the file path and permissions to ensure that Choreo can access the file. 

If you need further assistance, feel free to reach out to our support team for help.

In [61]:
# now let's load all the Choreo docs and store them in a vector db 
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[("#", "Header1"), ("##", "Header2"), ("###", "Header3")])

sections = []
files = load_files("data/")
for file in files:
    document = file[1].read()
    sections += markdown_splitter.split_text(document)

len(sections), sections[4:7]

(705,
 [Document(page_content='An organization is a logical grouping of users and their resources. It may represent a company, community, or a single user. Users can belong to multiple organizations, and each organization can have different roles assigned to its users to control access to Choreo features.', metadata={'Header1': 'Frequently Asked Questions', 'Header2': 'General', 'Header3': 'Q: What is an organization in Choreo?'}),
  Document(page_content='A project is a logical grouping of related components to help you organize your work. Each project provides runtime isolation through namespaces when you deploy components.', metadata={'Header1': 'Frequently Asked Questions', 'Header2': 'General', 'Header3': 'Q: What is a project in Choreo?'}),
  Document(page_content='A component is a workload designed to run on Choreo. Examples of components include integrations, APIs, microservices, manual/scheduled jobs, web apps, and triggers.', metadata={'Header1': 'Frequently Asked Questions',

In [65]:
# store embeddings in the database, if not exists 
embeddings = OpenAIEmbeddings(model = "text-embedding-3-small")
if not os.path.exists(DB_PATH):
    db = Chroma.from_documents(sections, embeddings, persist_directory=DB_PATH)
else:
    db =  Chroma( persist_directory=DB_PATH, embedding_function=embeddings)

In [75]:
docs = db.similarity_search(question, k = 3)

In [109]:
prompt_template = """You are an assistant to the Choreo users, to help them to use Choreo platform by using the given information to answer their questions.

Information: {context}

Question: {question}
"""
prompt = prompt_template.format(context = docs, question = question)

display(Markdown(prompt))

You are an assistant to the Choreo users, to help them to use Choreo platform by using the given information to answer their questions.

Information: [Document(page_content='The `endpoints.yaml` file has a specific structure and contains the following details:  \n| Field                | Required     | Description                                                                      |\n|----------------------|--------------|----------------------------------------------------------------------------------|\n| **version**          | Required     | The version of the `endpoints.yaml` file.                                           |\n| **name**             | Required     | A unique name for the endpoint, which Choreo will use to generate the managed API.|\n| **port**             | Required     | The numeric port value that gets exposed via this endpoint.                      |\n| **type**             | Required     | The type of traffic this endpoint is accepting, such as `REST`, `GraphQL`, `gRPC`, `UDP`or `TCP`. Currently, the MI preset supports only the `REST` type.                                         |\n| **networkVisibility**| Required     | The network level visibility of this endpoint, which defaults to `Project` if not specified. Accepted values are `Project`, `Organization`, or `Public`.|\n| **context**          | Required     | The context (base path) of the API that Choreo exposes via this endpoint.        |\n| **schemaFilePath**   | Required     |  The swagger definition file path. Defaults to the wildcard route if not provided. This field should be a relative path to the project path when using the **Java**, **Python**, **NodeJS**, **Go**, **PHP**, **Ruby**, and **WSO2 MI** buildpacks. For REST endpoint types, when using the **Ballerina** or **Dockerfile** buildpack, this field should be a relative path to the component root or Docker context.|  \n#### Sample endpoints.yaml  \n**File location**:  \n```bash\n<docker-build-context-path>/.choreo/endpoints.yaml\n```  \n!!! note\n- For components built with Ballerina buildpack `docker-build-context-path` should be replaced with `component-root`.\nFor example: `<component-root>/.choreo/endpoints.yaml`  \n- For components built with WSO2 MI buildpack `docker-build-context-path` should be replaced with `<Project Path>`.\nFor example: `<Project Path>/.choreo/endpoints.yaml`  \n**File content**:  \n```yaml\n# +required Version of the endpoint configuration YAML\nversion: 0.1\n\n# +required List of endpoints to create\nendpoints:\n# +required Unique name for the endpoint. (This name will be used when generating the managed API)\n- name: Greeting Service\n# +required Numeric port value that gets exposed via this endpoint\nport: 9090\n# +required Type of the traffic this endpoint is accepting. Example: REST, GraphQL, etc.\n# Allowed values: REST, GraphQL, GRPC, UDP, TCP\ntype: REST\n# +optional Network level visibility of this endpoint. Defaults to Project\n# Accepted values: Project|Organization|Public.\nnetworkVisibility: Project\n# +optional Context (base path) of the API that is exposed via this endpoint.\n# This is mandatory if the endpoint type is set to REST or GraphQL.\ncontext: /greeting\n# +optional Path to the schema definition file. Defaults to wild card route if not provided\n# This is only applicable to REST endpoint types.\n# The path should be relative to the docker context.\nschemaFilePath: greeting_openapi.yaml\n```', metadata={'Header2': 'Configure endpoints', 'Header3': 'Learn the endpoints.yaml file'}), Document(page_content='The method of defining endpoints depends on the buildpack. For buildpacks other than `Ballerina` and `WSO2 MI`, it is required to have an `endpoints.yaml` file in project root directory to create the Service component.', metadata={'Header2': 'Configure endpoints'}), Document(page_content='{% include "configure-endpoints-body.md" %}', metadata={'Header1': 'Configure Endpoints'})]

Question: how to fix endpoint.yaml missing error


In [91]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo')
answer = llm.invoke(prompt).content

display(Markdown(answer))

To fix the `endpoint.yaml` missing error, you need to ensure that you have created an `endpoints.yaml` file in the correct location and with the correct structure.

Here are the steps to fix the error:
1. Create an `endpoints.yaml` file in the project root directory or the specified location based on the buildpack being used.
2. Make sure the `endpoints.yaml` file contains the required fields such as `version`, `name`, `port`, `type`, `networkVisibility`, `context`, and `schemaFilePath`.
3. Verify that the values in the `endpoints.yaml` file are correctly formatted and follow the specified guidelines in the documentation.
4. Save the `endpoints.yaml` file and restart the Choreo platform to apply the changes.

By following these steps and ensuring that the `endpoints.yaml` file is correctly configured, you should be able to resolve the `endpoint.yaml` missing error.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

In [110]:
prompt = """You are an assistant to the Choreo users, to help them to use Choreo platform by using the given information to answer their questions.

Information: {context}

Question: {question}
"""

prompt_template = ChatPromptTemplate.from_messages([HumanMessagePromptTemplate.from_template(prompt)])
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": db.as_retriever() | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [111]:
answer = rag_chain.invoke(question)
display(Markdown(answer))

To fix the `endpoints.yaml` missing error, you need to create an `endpoints.yaml` file with the correct structure and details in the specified location within your project. Here is a step-by-step guide to create the `endpoints.yaml` file:

1. Create a new file named `endpoints.yaml` in your project's root directory or in the `.choreo` directory at the build context path.
2. Copy and paste the sample `endpoints.yaml` content provided in the information section above.
3. Make sure to fill in the required fields such as `version`, `name`, `port`, `type`, `networkVisibility`, `context`, and `schemaFilePath` with appropriate values specific to your endpoint configuration.
4. Save the `endpoints.yaml` file.

By following these steps and providing the necessary endpoint details in the `endpoints.yaml` file, you should be able to resolve the `endpoints.yaml` missing error. Remember to commit the file to your source repository before building the service component.

In [112]:
answer = rag_chain.invoke("how to learn more about Generative AI?")
display(Markdown(answer))

To learn more about Generative AI, you can explore online resources, research papers, attend workshops or webinars, and enroll in courses or programs specifically focused on Generative AI. Additionally, you can join forums or communities related to AI where experts and enthusiasts share knowledge and insights about Generative AI. Stay updated with the latest developments in the field by following reputable sources, such as research institutions, AI conferences, and industry publications.

In [119]:
prompt = """You are an assistant to the Choreo users, to help them to use Choreo platform by using the given information to answer their questions. DO NOT answer the questions unrelated to Choreo platform.

Choreo is an internal developer platform that allows build and deploying (CI/CD), observe, manage user services, proxies, web-apps etc

Information: {context}

Question: {question}
"""

prompt_template = ChatPromptTemplate.from_messages([HumanMessagePromptTemplate.from_template(prompt)])
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": db.as_retriever() | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

answer = rag_chain.invoke("how to learn more about Generative AI?")
display(Markdown(answer))

I can only provide information related to the Choreo platform. If you have any questions about Choreo or need help with tasks related to Choreo, feel free to ask!